#  Лабораторная работа №1 «Дискретизация аналоговых сигналов»



Курс: Радиофизическая лаборатория (ФРКТ МФТИ).

LMS: https://lms.mipt.ru/course/view.php?id=1569&type=lab
* В имени этого файла заменить familyname на Ваши фамилию и имя, либо только фамилию.  

# Занятие 1. Классификация сигналов: аналоговые, дискретные, цифровые.

In [ ]:
import numpy as np                # импорт бибилиотеки numpy
import matplotlib.pyplot as plt   # импорт модуля matplotlib.pyplot
import scipy.io.wavfile # импорт модуля scipy.io.wavfile

: 

In [ ]:
# Для использования backend matplotlib inline/widget/notebook, раскомментируйте соответствующую строку ниже.
# Backend notebook используется в старых версиях. 
# Установка widget: pip install --upgrade jupyterlab ipympl
%matplotlib inline
# %matplotlib notebook
# %matplotlib widget

In [ ]:
def quantize_uniform(x, quant_min=-1, quant_max=1, quant_level=5):
    """Uniform quantization approach
    From: Müller M. Fundamentals of music processing: Audio, analysis, algorithms, applications. – Springer, 2015.
    Notebook: C2S2_DigitalSignalQuantization.ipynb
    Args:
        x: Original signal
        quant_min: Minimum quantization level
        quant_max: Maximum quantization level
        quant_level: Number of quantization levels
    Returns:
        x_quant: Quantized signal
    """
    x_normalize = (x-quant_min) * (quant_level-1) / (quant_max-quant_min)
    x_normalize[x_normalize > quant_level - 1] = quant_level - 1
    x_normalize[x_normalize < 0] = 0
    x_normalize_quant = np.around(x_normalize)
    x_quant = (x_normalize_quant) * (quant_max-quant_min) / (quant_level-1) + quant_min
    return x_quant

In [ ]:
import scipy.integrate as integrate # импорт модуля численного интегрирования
def integrate_function(t, func, f, tau, real_part=True):
    # Подынтегральное выражение для использованиия в функции integrate.quad
    # t - время
    # func - функция, задающая импульс
    # f - частота
    # tau - константа, используемая для задания длительности импульса
    if real_part:
        return func(t, tau)*np.cos(-2*np.pi*f*t)  # действительная часть интеграла
    else:
        return func(t, tau)*np.sin(-2*np.pi*f*t)  # мнимая часть интеграла
def fourier_transform(signal, f_band, tau, t1, t2, res_type='abs'):
    # вычисление преобразования Фурье для набора частот
    # signal - функция от t и tau, задающая сигнал во временной области 
    # f_band - набор частот, для которых вычисляется преобразование Фурье
    # tau - константа, используемая для задания длительности импульса
    # t1 момент начала сигнала
    # t2 момент завершения сигнала
    # тип возвращаемого значения:
    # res_type='abs' - |X(f)|
    # res_type='Re' - Re X(f)
    # res_type='Im' - Im X(f)
    if res_type=="abs":
        Re=np.array([integrate.quad(integrate_function, t1, t2, args=(signal, f, tau, True))[0] for f in f_band])
        Im=np.array([integrate.quad(integrate_function, t1, t2, args=(signal, f, tau, False))[0] for f in f_band])
        return abs(Re+1j*Im)
    elif res_type=="Re":
        Re=np.array([integrate.quad(integrate_function, t1, t2, args=(signal, f, tau, True))[0] for f in f_band])
        return Re
    elif res_type=="Im":
        Im=np.array([integrate.quad(integrate_function, t1, t2, args=(signal, f, tau, False))[0] for f in f_band])
        return Im

## Задача 0. Введение в Python 3. (Задание к допуску)

1) Подгрузить библиотеку *Numpy* (`import numpy as np`).

2) Создать массив от 1 до 10 с шагом 2 – [1, 3, 5, 7, 9] – тремя разными способами (используя  три разные функции `np.array()`, `np.arange()`, `np.linspace()`).

3) Создать массив целых чисел от 0 до 100 (размерностью 100 элементов) максимально короткой записью.

4) Создать массив `k`, такой же как описан в п.3. Используя его, создать массив (набор отсчетов) `x`, соответствующий формуле $x[k] = \sin(2\pi \dfrac{f_0}{f_s}k)$, где $\dfrac{f_0}{f_s}$ взять равным 0.07. Следует использовать функцию `np.sin()` и константу `np.pi`.

5) Подгрузить модуль *pyplot* библиотеки *Matplotlib* (`import matplotlib.pyplot as plt`).

6) Создать холст (другие названия полотно, канва) размером 12 на 5 дюйма (`figsize = [12, 5]`).

7) Построить график $x[k]$ (`plt.plot(k, x)`), сделать подписи осей и заголовок графика.

8) Поменять цвет линии графика из п.7 на красный или оранжевый и тип линии на пунктир. Построить на графике отсчеты $x[k]$ с помощью функции `plt.stem(k, x)`, см. Задача 1.1 и Задача 1.2.  
(Для того чтобы поменять тип линии на пунктир, нужно в параметре отвечающем за цвет поставить модификатор "--", например `'--b'`.)

9) Работа с функциями.
  1. Создать функцию, соответствующую формуле $\dfrac{\sin(x)}{x}$. `def my_fun(x):`
  2. Незабыть обработать неопределенность в 0.  
  `if x=0:`  
  ....`return 1`  
  `else`  
  ....`return np.sin(x)/x`
  3. Выполнить п.4 и п.7, создав новый массив значений `y`, используя созданную вами функцию.

10)  Выполнить п.4, только для массива `z`, который соответствует комплексной экспоненциальной функции $z[k] = \exp(-j\,2\pi\,\dfrac{f_0}{f_s}k)$  (функция `np.exp()`). Выполнить п.7, но построить отдельно график реальной и график мнимой частей этой функции.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

arr_a = np.array([1, 3, 5, 7, 9])
arr_b = np.arange(1, 10, 2)
arr_c = np.linspace(1, 9, 5, dtype=int)

k = np.arange(100)
ratio = 0.07
x = np.sin(2 * np.pi * ratio * k)

plt.figure(figsize=(12, 5))
plt.plot(k, x)
plt.xlabel('k')
plt.ylabel('x[k]')
plt.title('x[k] = sin(2π * 0.07 * k)')
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(k, x, 'r--', label='r-- (пунктир, красный)')
plt.stem(k, x, basefmt=" ", label='отсчеты (stem)')
plt.xlabel('k')
plt.ylabel('x[k]')
plt.title('x[k] — пунктирная линия и отсчеты (stem)')
plt.legend()
plt.grid(True)
plt.show()

def my_fun(x):
    x = np.asarray(x)
    return np.where(x == 0, 1.0, np.sin(x) / x)

y = my_fun(2 * np.pi * ratio * k)

plt.figure(figsize=(12, 5))
plt.plot(k, y, 'b-')
plt.xlabel('k')
plt.ylabel('y[k] = sin(θ)/θ, θ=2π·0.07·k')
plt.title('y[k] = sinc-like функция my_fun(2π·0.07·k)')
plt.grid(True)
plt.show()

z = np.exp(-1j * 2 * np.pi * ratio * k)

plt.figure(figsize=(12, 5))
plt.plot(k, z.real, label='Re{z[k]}')
plt.xlabel('k')
plt.ylabel('Re{z[k]}')
plt.title('Реальная часть z[k] = exp(-j·2π·0.07·k)')
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(k, z.imag, label='Im{z[k]}')
plt.xlabel('k')
plt.ylabel('Im{z[k]}')
plt.title('Мнимая часть z[k] = exp(-j·2π·0.07·k)')
plt.grid(True)
plt.legend()
plt.show()

## Задача 1.1. Дискретизация и квантование.

| Варианты: |   1 |    2 |    3 |   4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |
|:---------:|----:|-----:|-----:|----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|     **N**     |  15 |   20 |   30 |  12 |   15 |   30 |   25 |   18 |   10 |   25 |   20 |   15 |
|   $f_0$**, Гц**   | 200 |  250 |  400 | 200 |  600 |  750 | 1200 |  600 |  400 |  500 |  800 |  400 |
|   $f_s$**, Гц**   | 500 | 1250 | 2000 | 600 | 1500 | 3750 | 6000 | 1800 | 1000 | 2500 | 4000 | 1200 |

Промоделировать дискретизацию и квантование синусоидального сигнала $x(t)=\sin(2\pi f_0 t)$ с частотой дискретизации $f_s$. Пронаблюдать эффекты во временной области для отрезка сигнала из $N$ отсчетов. 

1) Как соотносится растояние между отсчетами по оси времени с частотой дискретизации?

**Ответ:**

Расстояние между соседними отсчётами по оси времени:
$$
\Delta t=\frac{1}{f_s},\qquad t_k=\frac{k}{f_s}\;(k=0,1,\dots,N-1),
$$

длительность отрезка из $N$ отсчётов равна $(N-1)/f_s$ (приблизительно $N/f_s$ при большом $N$).


2) Как изменяется максимально возможная абсолютная ошибка квантования $\varepsilon[k]=|x[k]-y[k]|$ с ростом числа уровней квантования?

**Ответ:**

При равномерном квантовании на отрезке $[q_{\min},q_{\max}]$ шаг квантования
$$
\Delta=\frac{q_{\max}-q_{\min}}{L},
$$
где $L$ — число уровней. Теоретическая максимальная абсолютная ошибка
$$
\varepsilon_{\max}=\frac{\Delta}{2}=\frac{q_{\max}-q_{\min}}{2L}.
$$

Поэтому с увелечением L ошибка уменьшится.



In [ ]:
N=25                                    # Число отсчетов по времени
f0=200.0                                # Частота синусоиды в Гц
fs=2000.0                               # Частота дискретизации в Гц
k=np.arange(N)                          # Mассив времен k от 0 до N-1 с шагом 1
x=np.sin(2*np.pi*(f0/fs)*k)             # Последовательность x[k]
num_levels=8                      # число уровней квантования

In [ ]:
# Построение графиков аналогового и дискретизованного сигнала
plt.figure(figsize=[8, 4])              # создание полотна размером шириной 8 X 4 дюйма
t=np.linspace(0, N/fs, num=1024)        # создание массива времен t (1024 значения от 0 до N*fs)

plt.plot(t, np.sin(2*np.pi*f0*t), 'g', label='аналоговый сигнал $x(t)$')  
                                        # построение графика x(t) (точки соединяются линиями)
                                        # 'g' означает, что используется зеленая линия (green)
plt.stem(k/fs, x, linefmt='b', markerfmt='bo', label='дискретизованный сигнал $x[k]$')            
                                        # построение графика функции дискретного времени x[k]
                                        # 'b', 'bo' означает, что отсчеты оборажаются синим цветом (blue)
plt.grid()                              # сетка
plt.xlabel("$t$, c")                    # подпись оси X
plt.ylabel("$x(t), x[k]$")              # подпись оси Y
plt.title("Аналоговый и дискретизованный сигналы") # заголовок графика
plt.legend(loc='best', bbox_to_anchor=(1, -0.1))
plt.tight_layout()                      # автоматическая корректировка расположения осей графика
plt.show()

In [ ]:
# моделирование квантования дискретного сигнала
y=quantize_uniform(x, quant_min=-1, quant_max=1, quant_level=num_levels)
bins=np.linspace(-1, 1, num_levels)

In [ ]:
# Вывод графика аналогового и цифрового сигнала
plt.figure(figsize=[8, 4])                  # создание полотна размером шириной 8 X 4 дюйма
t=np.linspace(0, N/fs, num=1024)            # создание массива времен t (1024 значения от 0 до N*fs)
plt.plot(t, np.sin(2*np.pi*f0*t), 'g', label='аналоговый сигнал $x(t)$')        
                                            # построение графика x(t) (точки соединяются линиями)

plt.stem(k/fs, y, linefmt='b', markerfmt='bo', label='цифровой сигнал $y[k]$')                
                                            # построение графика функции дискретного времени y[k]
                                            # k/fs - м
    
if num_levels<21:                           # если число уровней не велико, то производится
    plt.yticks(bins)                        # установка делений шкалы оси Y, совпадающих с уровнями квантования

plt.grid()                                  # сетка
plt.xlabel("$t$, c")                        # подпись оси X
plt.ylabel("$x(t), y[k]$")                  # подпись оси Y
plt.title("Аналоговый и цифровой сигналы")  # заголовок графика
plt.legend(loc='best', bbox_to_anchor=(1, -0.1))
plt.tight_layout()                          # автоматическая корректировка расположения осей графика
plt.show()

In [ ]:
abs(x-y)

In [ ]:
plt.figure(figsize=[8, 4])
plt.title("Ошибка квантования") 
plt.plot(t, np.sin(2*np.pi*f0*t), 'g', label='аналоговый сигнал $x(t)$')
plt.stem(k/fs, abs(x-y), linefmt='m', markerfmt='mo', label="абсолютная ошибка \nквантования \n$\\epsilon[k]=|x[k]-y[k]|$")       
                                      #вывод абсолютных значений ошибки квантования для дискретного сигнала
plt.yticks(bins)
plt.grid()
plt.xlabel("$t$, c")                                          # подпись оси X
plt.ylabel("$x(t), \\varepsilon[k]=|x[k]-y[k]|$")             # подпись оси Y
plt.legend(loc='best', bbox_to_anchor=(1, -0.1))
plt.tight_layout() 
plt.show()

## Задача 1.2. Декодирование .wav файла

| Варианты: |       1      |     2     |         3        |      4     |        5        |        6       |         7        |       8      |        9       |     10    |        11       |     12     |
|:---------:|:------------:|:---------:|:----------------:|:----------:|:---------------:|:--------------:|:----------------:|:------------:|:--------------:|:---------:|:---------------:|:----------:|
| **Имя файла** | clarinet.wav | flute.wav | glockenspiel.wav | guitar.wav | tuning-fork.wav | vibraphone.wav | glockenspiel.wav | clarinet.wav | vibraphone.wav | flute.wav | tuning-fork.wav | guitar.wav |

Декодируйте .wav файл в соотвествии с Вашим вариантом из таблицы. 

1) Определите частоту дискретизации, битную глубину записи (соотвествует dtype) и соотвествующее ей колличество уровней квантования. 

2) Сравните растояние между отсчетами по оси времени на графике с частотой дискретизации.

3) Поменяйте ось времени (первый аргумент функции `plt.plot`) так, чтобы первому отсчету наблюдаемого диапазона соотвествовало время от начала файла.

4) Определите длину записи в секундах.

5) Оцените объем файла в килобайтах (КБ), при условии, что никакое дополнительное сжатие не производится (использовать данные о частоте дискретизации, числе отсчетов и битовой глубине). Сравние с реальным объемом файла. 

In [ ]:
from IPython.display import Audio
Audio('tuning-fork.wav')

In [ ]:
fs, x = scipy.io.wavfile.read('tuning-fork.wav') # чтение аудиофайла 

In [ ]:
fs     #частота дискретизации в Гц

In [ ]:
x      #массив значений x[k]

In [ ]:
print(x.dtype) #тип данных для значений x[k]

In [ ]:
x.size #размер массива

In [ ]:
x1=x[8000:10000]                     # выбор наблюдаемого диапазона
k=np.arange(x1.size)               # отсчеты по времени
# Построение графиков 
plt.figure(figsize=[8, 4])         # создание полотна размером шириной 8 X 4 дюйма
plt.plot(k/fs, x1, 'b.')           # построение графика цифрового сигнала точками точками
plt.grid()                             
plt.xlabel("$t$, c")                      
plt.ylabel("$x[k]$")             
plt.tight_layout()
plt.show()

Программа для ответа на вопросы:

In [ ]:
import os
import wave
import glob
import numpy as np
import scipy.io.wavfile as wavfile
import matplotlib.pyplot as plt

audio_dir = "audio"
plot_out_dir = "audio_plots"
os.makedirs(plot_out_dir, exist_ok=True)

windows = [
    (8000, 10000),
    (8000, 10000),
    (8000, 10000),
    (8000, 10000),
    (8000, 10000),
    (8000, 10000)
]

wav_paths = sorted(glob.glob(os.path.join(audio_dir, "*.wav")))

for i, path in enumerate(wav_paths):
    name = os.path.basename(path)
    base, _ = os.path.splitext(name)

    start, end = windows[i % len(windows)]

    with wave.open(path, "rb") as wf:
        nchannels = wf.getnchannels()
        sampwidth = wf.getsampwidth()
        framerate  = wf.getframerate()
        nframes    = wf.getnframes()
        comptype   = wf.getcomptype()

    bits_per_sample = sampwidth * 8

    fs, x = wavfile.read(path)

    if x.ndim == 1:
        x = x.reshape(-1, 1)

    n_samples = x.shape[0]
    channels = x.shape[1]

    start = int(start)
    end = int(end)
    if start < 0:
        start = 0
    if end <= start:
        end = min(start + 1, n_samples)
    if start >= n_samples:
        start = 0
        end = min(1, n_samples)
    if end > n_samples:
        end = n_samples

    x_window = x[start:end, 0]
    k_local = np.arange(x_window.size)

    k_window = np.arange(start, end)
    times_window = k_window / fs
    dt_mean = float(np.mean(np.diff(times_window))) if len(times_window) > 1 else None

    duration_seconds = nframes / fs
    estimated_bytes = nframes * channels * (bits_per_sample // 8)
    estimated_kb = estimated_bytes / 1024.0
    actual_bytes = os.path.getsize(path)
    actual_kb = actual_bytes / 1024.0
    ratio = (actual_kb / estimated_kb) if estimated_kb > 0 else float('nan')

    print(f"\nfile: {name}")
    print(f"applied window #{i % len(windows)}: start={start}, end={end} (samples)")
    print(f"fs = {fs} Hz, channels = {channels}, sampwidth = {sampwidth} байт, bits_per_sample = {bits_per_sample}")
    print(f"dtype scipy = {x.dtype}", end="")
    print(f", bits_from_dtype = {x.dtype.itemsize * 8}, levels = {2 ** (x.dtype.itemsize * 8)}")
    print(f"nframes = {nframes}, duration_seconds = {duration_seconds:.6f} s")
    print(f"средний шаг в окне dt_mean = {dt_mean} s, 1/fs = {1.0/fs} s, разница = {None if dt_mean is None else dt_mean - 1.0/fs}")
    print(f"estimated size= {estimated_kb:.2f} kb, actual_kb = {actual_kb:.2f} kb, ratio = {ratio:.3f}")

    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(k_local / fs, x_window, '.')
    plt.title(f"{base}: k_local/fs")
    plt.xlabel("t, s")
    plt.grid(True)

    plt.subplot(1,2,2)
    t_correct = (start + k_local) / fs
    plt.plot(t_correct, x_window, '.')
    plt.title(f"{base}: время от начала файла")
    plt.xlabel("t, s")
    plt.grid(True)

    out_png = os.path.join(plot_out_dir, f"{base}_analysis.png")
    plt.tight_layout()
    plt.savefig(out_png, dpi=150)
    plt.close()


Вывод программы:

```
file: clarinet.wav
applied window #0: start=8000, end=10000 (samples)
fs = 44100 Hz, channels = 1, sampwidth = 2 байт, bits_per_sample = 16
dtype scipy = int16, bits_from_dtype = 16, levels = 65536
nframes = 183808, duration_seconds = 4.167982 s
средний шаг в окне dt_mean = 2.2675736961451248e-05 s, 1/fs = 2.2675736961451248e-05 s, разница = 0.0
estimated size= 359.00 kb, actual_kb = 363.00 kb, ratio = 1.011

file: flute.wav
applied window #1: start=8000, end=10000 (samples)
fs = 48000 Hz, channels = 1, sampwidth = 4 байт, bits_per_sample = 32
dtype scipy = int32, bits_from_dtype = 32, levels = 4294967296
nframes = 528000, duration_seconds = 11.000000 s
средний шаг в окне dt_mean = 2.0833333333333343e-05 s, 1/fs = 2.0833333333333333e-05 s, разница = 1.0164395367051604e-20
estimated size= 2062.50 kb, actual_kb = 2062.54 kb, ratio = 1.000

file: glockenspiel.wav
applied window #2: start=8000, end=10000 (samples)
fs = 96000 Hz, channels = 1, sampwidth = 4 байт, bits_per_sample = 32
dtype scipy = int32, bits_from_dtype = 32, levels = 4294967296
nframes = 129373, duration_seconds = 1.347635 s
средний шаг в окне dt_mean = 1.0416666666666671e-05 s, 1/fs = 1.0416666666666666e-05 s, разница = 5.082197683525802e-21
estimated size= 505.36 kb, actual_kb = 505.41 kb, ratio = 1.000

file: guitar.wav
applied window #3: start=8000, end=10000 (samples)
fs = 44100 Hz, channels = 1, sampwidth = 2 байт, bits_per_sample = 16
dtype scipy = int16, bits_from_dtype = 16, levels = 65536
nframes = 639450, duration_seconds = 14.500000 s
средний шаг в окне dt_mean = 2.2675736961451248e-05 s, 1/fs = 2.2675736961451248e-05 s, разница = 0.0
estimated size= 1248.93 kb, actual_kb = 1248.98 kb, ratio = 1.000

file: tuning-fork.wav
applied window #4: start=8000, end=10000 (samples)
fs = 44100 Hz, channels = 1, sampwidth = 2 байт, bits_per_sample = 16
dtype scipy = int16, bits_from_dtype = 16, levels = 65536
nframes = 135680, duration_seconds = 3.076644 s
средний шаг в окне dt_mean = 2.2675736961451248e-05 s, 1/fs = 2.2675736961451248e-05 s, разница = 0.0
estimated size= 265.00 kb, actual_kb = 265.04 kb, ratio = 1.000

file: vibraphone.wav
applied window #5: start=8000, end=10000 (samples)
fs = 44100 Hz, channels = 1, sampwidth = 2 байт, bits_per_sample = 16
dtype scipy = int16, bits_from_dtype = 16, levels = 65536
nframes = 143336, duration_seconds = 3.250249 s
средний шаг в окне dt_mean = 2.2675736961451248e-05 s, 1/fs = 2.2675736961451248e-05 s, разница = 0.0
estimated size= 279.95 kb, actual_kb = 280.00 kb, ratio = 1.000
```

# Занятие 2. Спектры импульсных и периодических сигналов.

## Задача 2.1. Спектры симметричных оконных функций

|  Варианты:  |  1  |  2  |   3  |   4  |  5  |   6  |  7  |   8  |   9  |  10 |  11  |  12  |
|:-----------:|:---:|:---:|:----:|:----:|:---:|:----:|:---:|:----:|:----:|:---:|:----:|:----:|
| $\tau$**, мкс** | 100 | 500 | 1000 | 5000 | 400 | 2000 | 300 | 1500 | 3000 | 800 | 3500 | 4000 |


Ниже приведен пример вычисления модуля спектральной плотности $|X(f)|$ для прямоугольного окна длительностью $\tau=150$ мкс. 

Для заданного параметра $\tau$ из таблицы, построить графики модуля спектральной плотности $|X(f)|$ для прямоугольного окна, 
$$w_b(t)=\left\{ \begin{matrix}
   1, & \text{ }\text{если }|t|\le \tau /2,  \\
   0, & \text{ }\text{если }|t|>\tau /2; \\
\end{matrix} \right.$$
треугольного окна
$$w_t(t)=\left\{ \begin{matrix}
   1-\dfrac{|t|}{\tau /2}, & \text{ }\text{если }|t|\le \tau /2,  \\
   0, & \text{ }\text{если }|t|>\tau /2.  \\
\end{matrix} \right.$$
 и окна Ханна
$$w_H(t)=\left\{ \begin{matrix}
   \dfrac{1}{2}\left( 1+\cos \left( 2\pi \dfrac{t}{\tau } \right) \right), & \text{ }\text{если }|t|\le \tau /2,  \\
   0, & \text{ }\text{если }|t|>\tau /2.  \\
\end{matrix} \right.$$


Привести аналитические формулы спектральной плотности для этих импульсов. 
Определить 
* а) уровни первого бокового лепестка относительно главного в $|X(f)|$ (практич.) в дБ ($20\lg  \left|\dfrac{X(f)}{X(0)}\right| $ ),
* б) расположение нулей $|X(f)|$ (практич. и теоретическое).

Заполнить таблицу для трех окон.

| Окна  |   Уровень первого бокового лепестка относительно главного, дБ  |  Ширина главного лепестка, Гц  |
|:-----------:|:---:|:---:|
| прямоугольное |-13.27|6655|
| треугольное |-26.52|13353|
| Ханна |-31.47|6683|

### **Аналитические формулы спектральной плотности**

**Формула:**
$$ X(f) = \int_{-\tau/2}^{\tau/2} w(t)e^{-j2\pi f t}\,dt $$

**1. Прямоугольное окно**
$$
X_b(f)=\int_{-\tau/2}^{\tau/2} e^{-j2\pi f t}\,dt
      = \tau\,\mathrm{sinc}(\pi f\tau),
\qquad \mathrm{sinc}(x)=\frac{\sin x}{x}.
$$

**2. Треугольное окно**
(высота 1, база $\tau$)
$$
X_t(f)=\frac{\tau}{2}\,\mathrm{sinc}^2\!\left(\frac{\pi f\tau}{2}\right),
$$
так как площадь треугольника $\tau/2$, $X_t(0)=\tau/2$.

**3. Окно Ханна**
$$
w_H(t)=\frac{1}{2}\left(1+\cos\left(\frac{2\pi t}{\tau}\right)\right),
\quad |t|\le\tau/2.
$$

$$
X_H(f)=\frac{\tau}{2}\,\mathrm{sinc}(\pi f\tau)
+\frac{\tau}{4}\Big[\mathrm{sinc}\big(\pi(f\tau-1)\big)
+\mathrm{sinc}\big(\pi(f\tau+1)\big)\Big].
$$
Нормировка выбрана так, что $X_H(0)=\tau/2$.

Код для построения графиков для трех разных форм:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

tau = 150e-6
f = np.linspace(-8/tau, 8/tau, 2000)

sinc = lambda x: np.where(x==0.0, 1.0, np.sin(x)/x)

Xb = tau * sinc(np.pi * f * tau)                      # прямоугольное
Xt = (tau/2) * (sinc(np.pi * f * tau / 2))**2         # треугольное
Xh = (tau/2)*sinc(np.pi*f*tau) + (tau/4)*(sinc(np.pi*(f*tau-1)) + sinc(np.pi*(f*tau+1)))  # хан

def to_dB(X):
    X0 = np.abs(X[f==0])[0] if np.any(f==0) else np.abs(X).max()
    return 20*np.log10(np.abs(X)/X0 + 1e-20)

plt.figure()
plt.plot(f/1e3, np.abs(Xb), label='rect |X|')
plt.plot(f/1e3, np.abs(Xt), label='tri |X|')
plt.plot(f/1e3, np.abs(Xh), label='hann |X|')
plt.xlim(-40, 40)  # кГц
plt.xlabel('f, kHz'); plt.ylabel('|X(f)|'); plt.legend(); plt.grid()

plt.figure()
plt.plot(f/1e3, to_dB(Xb), label='rect dB')
plt.plot(f/1e3, to_dB(Xt), label='tri dB')
plt.plot(f/1e3, to_dB(Xh), label='hann dB')
plt.xlim(0, 60); plt.ylim(-80, 5)
plt.xlabel('f, kHz'); plt.ylabel('20 log10(|X/X(0)|), dB'); plt.legend(); plt.grid()
plt.show()


In [ ]:
tau=100e-6 #100 мкс

f_band=np.linspace(-8/tau, 8/tau, 500) # 500 - число точек в диапазоне, в которых вычисляется X(f)

def boxcar(t, tau):
    if abs(t)<tau/2:
        return 1.0                 
    else:
        return 0.0                 
    
t_band=np.linspace(-2*tau, 2*tau, 1024)
plt.figure(figsize=[6, 4])
plt.plot(t_band*1e6, [boxcar(t, tau) for t in t_band])
plt.xlabel("Время $t$, мкс")
plt.ylabel("$x(t)$, В")
plt.title("Сигнал")
plt.tight_layout() 
plt.grid()

plt.figure(figsize=[6, 4])
plt.plot(f_band/1e3, fourier_transform(signal=boxcar, f_band=f_band, tau=tau, t1=-tau/2, t2=tau/2, res_type="abs")*1e6)
plt.xlabel("Частота $f$, кГц")
plt.ylabel("$|X(f)|$,  мкВ / Гц")
plt.title("Спектр")
plt.tight_layout() 
plt.grid()
plt.show()

### Определение уровней первого бокового лепестка (практич.)
1) Прямоугольное: -13.27dB
2) Треугольное: -26.52dB
3) Хан: -31.47dB


### Определение нулей
**Практич.**
Прямоугольное: 6666.67 Гц
Треугольное: 13333.33 Гц
Хан: 13333.33 Гц

**Теор.**
Прямоугольное:
$1/{\tau} = 1 / {150 * 10^{-6}} = 6666.67 Гц$

Треугольное:
$2/{\tau} = 2 / {150 * 10^{-6}} = 13333.33 Гц$

Хан:
$2/{\tau} = 2 / {150 * 10^{-6}} = 13333.33 Гц$

Как видим, теоретические и эксперементальные данные совпали!


## Задача 2.2. Теорема запаздывания для преобразования Фурье

|  Варианты:  |  1  |  2  |   3  |   4  |  5  |   6  |  7  |   8  |   9  |  10 |  11  |  12  |
|:-----------:|:---:|:---:|:----:|:----:|:---:|:----:|:---:|:----:|:----:|:---:|:----:|:----:|
|     **E, В**    | 0.1 | 0.5 |   1  |   5  |  1  |   5  | 0.5 |  2.5 |   6  | 1.6 |   7  |   8  |
| **$\tau$, мкс** | 100 | 500 | 1000 | 5000 | 400 | 2000 | 300 | 1500 | 3000 | 800 | 3500 | 4000 |

Ниже приведен пример вычисления спектральной плотности для несимметричного прямоугольного импульса амплитудой $E=0.1$ В, длительностью $\tau=100$ мкс. В данном случае импульс начинается в начале отсчета оси времени. 

Для заданных параметров $E$ и $\tau$ из таблицы, построить и сравнить графики $\text{Re} X(f)$, $\text{Im} X(f)$, $|X(f)|$ для прямоугольного импульса, начинающегося в моменты времени $-\tau /2$ и $0$. Как согласуются результаты с теоремой запаздывания для преобразования Фурье?


In [ ]:
tau=150*1e-6 #150 мкс

f_band=np.linspace(-4/tau, 4/tau, 1000) # 1000 - число точек в диапазоне, в которых вычисляется X(f)

def boxcar(t, tau):
    if 0<t<tau:
        return 0.1 # 0.1 В
    else:
        return 0.0 # 0 В
    
t_band=np.linspace(-2*tau, 2*tau, 1024)
plt.figure(figsize=[8, 4])
plt.plot(t_band*1e6, [boxcar(t, tau) for t in t_band])
plt.xlabel("Время $t$, мкс")
plt.ylabel("$x(t)$, В")
plt.title("Сигнал")
plt.tight_layout() 
plt.grid()
plt.show()

plt.figure(figsize=[8, 4])
plt.plot(f_band/1e3, fourier_transform(signal=boxcar, f_band=f_band, tau=tau, t1=-2*tau, t2=2*tau, res_type="abs")*1e6, label='$|X(f)|$')
plt.plot(f_band/1e3, fourier_transform(signal=boxcar, f_band=f_band, tau=tau, t1=-2*tau, t2=2*tau, res_type="Re")*1e6, label='${Re }X(f)$')
plt.plot(f_band/1e3, fourier_transform(signal=boxcar, f_band=f_band, tau=tau, t1=-2*tau, t2=2*tau, res_type="Im")*1e6, label='${Im }X(f)$')
plt.xlabel("Частота $f$, кГц")
plt.ylabel("$|X(f)|$,  мкВ / Гц")
plt.title("Спектр")
plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.tight_layout() 
plt.grid()
plt.show()

Полный код для построения графиков:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

E = 0.1        
tau = 150e-6   

f_band = np.linspace(-4.0/tau, 4.0/tau, 1000) 

def sinc(x):
    z = np.zeros_like(x, dtype=float)
    small = np.abs(x) < 1e-16
    z[small] = 1.0
    z[~small] = np.sin(x[~small]) / x[~small]
    return z

X_common = E * tau * sinc(np.pi * f_band * tau)

X_center = X_common.copy()  

X_start0 = X_common * np.exp(-1j * np.pi * f_band * tau)

abs_center = np.abs(X_center)
abs_start0 = np.abs(X_start0)
re_center = np.real(X_center)
re_start0 = np.real(X_start0)
im_center = np.imag(X_center)
im_start0 = np.imag(X_start0)

max_abs_diff = np.max(np.abs(abs_center - abs_start0))

eps = 1e-20
mask = np.abs(X_center) > eps 
ratio = X_start0[mask] / X_center[mask]
theoretical = np.exp(-1j * np.pi * f_band[mask] * tau)
max_ratio_err = np.max(np.abs(ratio - theoretical))

phase_ratio = np.angle(ratio)               
phase_theor = np.angle(theoretical)
phase_ratio_un = np.unwrap(phase_ratio)
phase_theor_un = np.unwrap(phase_theor)
max_phase_diff = np.max(np.abs(phase_ratio_un - phase_theor_un))

plt.figure(figsize=(9,4))
plt.plot(f_band/1e3, abs_center, label='|X(f)|, центрированный (t∈[-τ/2,τ/2])')
plt.plot(f_band/1e3, abs_start0, '--', label='|X(f)|, начинается в t=0 (t∈[0,τ])')
plt.xlabel('f, кГц')
plt.ylabel('|X(f)|, В·с')
plt.title('Модуль спектра |X(f)| — сравнение (центр / старт в 0)')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()

plt.figure(figsize=(9,4))
plt.plot(f_band/1e3, re_center, label='Re X (центрированный)')
plt.plot(f_band/1e3, re_start0, '--', label='Re X (начинается в 0)')
plt.xlabel('f, кГц')
plt.ylabel('Re X(f), В·с')
plt.title('Действительная часть спектра Re X(f)')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()

plt.figure(figsize=(9,4))
plt.plot(f_band/1e3, im_center, label='Im X (центрированный)')
plt.plot(f_band/1e3, im_start0, '--', label='Im X (начинается в 0)')
plt.xlabel('f, кГц')
plt.ylabel('Im X(f), В·с')
plt.title('Мнимая часть спектра Im X(f)')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()

plt.figure(figsize=(9,4))
plt.plot(f_band[mask]/1e3, phase_ratio_un, label='arg(X_start0/X_center)')
plt.plot(f_band[mask]/1e3, phase_theor_un, '--', label='arg(exp(-jπ f τ))')
plt.xlabel('f, кГц')
plt.ylabel('Фаза, рад')
plt.title('Фазовая проверка: arg(X_start0/X_center) vs -π f τ')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()

plt.show()


# Согласование результатов с теоремой запаздывания

**Теорема запаздывания:**  
Если сигнал сдвигается во времени на $ t_0 $:
$$
x_{t_0}(t) = x(t - t_0),
$$
то его преобразование Фурье равно:
$$
X_{t_0}(f) = e^{-j2\pi f t_0} X(f).
$$

---

### Результаты эксперимента

- **Модули совпадают:**
  $$
  |X_{\text{start}}(f)| = |X_{\text{center}}(f)|.
  $$
  
  Сдвиг во времени не влияет на амплитудный спектр.

- **Фазовая разность линейна по частоте:**
  $$
  \arg\frac{X_{\text{start}}(f)}{X_{\text{center}}(f)} = -2\pi f t_0,
  $$
  где $ t_0 = \tau / 2 \Rightarrow \arg = -\pi f \tau $.

- **Численно:**
  - Разность модулей ≈ 0.  
  - Разность фаз с теоретической прямой $-\pi f \tau$ ≈ 0.  
  - Скачки фазы при нулях $|X(f)| \approx 0$ — нормальное поведение (фаза не определена в этих точках).

---

### **Вывод**
Результаты полностью согласуются с теоремой запаздывания:  
амплитуда спектра не изменяется, а фаза приобретает линейную зависимость $-2\pi f t_0$ (в данном случае — $-\pi f \tau$).


## Задача 2.3. Спектр отрезка синусоиды

|  Варианты:  |  1  |  2  |  3  |  4  |  5  |  6 |  7  |  8  |  9  | 10 |  11 |  12 |
|:-----------:|:---:|:---:|:---:|:---:|:---:|:--:|:---:|:---:|:---:|:--:|:---:|:---:|
|  $f_0$**, кГц** |  10 | 100 |  60 |  40 |  65 | 90 |  30 |  20 |  55 | 75 |  35 |  50 |
| $\tau$**, мкс** | 500 | 50  | 100 | 125 | 100 | 55 | 200 | 225 | 109 | 60 | 171 | 100 |

Ниже приведен пример вычисления спектральной плотности для отрезка синусоиды $x(t)=\sin(2\pi f_0 t)$ c частотой $f_0=10$ кГц длительностью  $\tau=500$  мкс. 

Для заданных параметров  $f_0$  и  $\tau$  из таблицы, построить $|X(f)|$ для отрезка синусоиды $x(t)=\sin(2\pi f_0 t)$ c частотой $f_0$ длительностью  $\tau$. 

1) Сравнить положение главных спектральных максимумов с частотой синусоиды. 

2) Повторить вычисления для отрезка синусоиды, взвешанного окном Ханна соотвествующей длины. Сравнить ширину главного лепестка на нулевом уровне и значение  $|X(f)|$ на частоте $f_0$ для сигнала $x(t)$, взвешанного прямоугольным окном и окном Ханна.

3) Сравнить спектр периодического сигнала  $x(t)=\sin(2\pi f_0 t)$ с получившимися графиками. 

In [ ]:
f0=10*1e3       # 10 кГц
tau=500*1e-6    # 500 мкс

def sin_with_boxcar(t, tau):                            
    if 0<=t<=tau:
        # случай прямоугольного окна
        return np.sin(2*np.pi*f0*t) 
        # случай окна Ханна
        # return np.sin(2*np.pi*f0*t) * (0.5+0.5*np.cos(np.pi*(t-tau/2)/(tau/2)))
    else:
        return 0.0
    
f_band=np.linspace(-2*f0, 2*f0, 2000) 

t_band=np.linspace(-0.5*tau, 1.5*tau, 1024)
plt.figure(figsize=[8, 4])
plt.plot(t_band*1e6, [sin_with_boxcar(t, tau) for t in t_band])
plt.title("Сигнал")
plt.xlabel("Время $t$, мкс")
plt.ylabel("$x(t)$, В")
plt.tight_layout() 
plt.grid()
plt.show()

plt.figure(figsize=[8, 4])
plt.plot(f_band/1e3, fourier_transform(signal=sin_with_boxcar, f_band=f_band, tau=tau, t1=-2*tau, t2=2*tau, res_type="abs")*1e6)
plt.title("Спектр")
plt.xlabel("Частота $f$, кГц")
plt.ylabel("$|X(f)|$,  мкВ / Гц")
plt.tight_layout() 
plt.grid()
plt.show()

Код для построения графиков:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

f0 = 10e3       
tau = 500e-6    

Nt = 10001
t = np.linspace(0, tau, Nt)

x_rect = np.sin(2*np.pi*f0*t)
w_hann = 0.5 + 0.5*np.cos(np.pi*(t - tau/2)/(tau/2))
x_hann = x_rect * w_hann

Nf = 2000
f_band = np.linspace(-2*f0, 2*f0, Nf)

def compute_FT(x, t, f_band):

    E = np.exp(-2j * np.pi * f_band[:, None] * t[None, :])
    X = np.trapz(x[None, :] * E, t, axis=1)
    return X

X_rect = compute_FT(x_rect, t, f_band)
X_hann = compute_FT(x_hann, t, f_band)

idx_f0 = np.argmin(np.abs(f_band - f0))
mag_rect_f0 = np.abs(X_rect[idx_f0])
mag_hann_f0 = np.abs(X_hann[idx_f0])

width_rect = 2.0 / tau
width_hann = 4.0 / tau

plt.figure(figsize=(8,3.5))
plt.plot(np.concatenate((t*1e6, (t+tau)*1e6))[:len(t)], x_rect, label='rect (время)')
plt.plot(t*1e6, x_hann, label='hann (время)', alpha=0.7)
plt.xlabel('t, мкс')
plt.ylabel('x(t)')
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.figure(figsize=(8,3.5))
plt.plot(f_band/1e3, np.abs(X_rect), label='|X| rect')
plt.plot(f_band/1e3, np.abs(X_hann), label='|X| hann', alpha=0.8)
plt.axvline(f0/1e3, color='k', linestyle='--', linewidth=0.6)
plt.axvline(-f0/1e3, color='k', linestyle='--', linewidth=0.6)
plt.xlabel('f, кГц')
plt.ylabel('|X(f)|')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Анализ спектральной плотности отрезка синусоиды

## 1) Положение главных спектральных максимумов

Главные максимумы модуля спектра $ |X(f)| $ располагаются вблизи частот  
$+f_0$ и $-f_0$, что совпадает с частотой исходной синусоиды.

---

## 2) Сравнение прямоугольного окна и окна Ханна

| Параметр | Прямоугольное окно | Окно Ханна |
|-----------|--------------------|-------------|
| Ширина главного лепестка (нулевой уровень) | $ 4\,\text{кГц} $ | $ 8\,\text{кГц} $ |
| Амплитуда $ X(f_0) $ | Максимальная | Примерно в 2 раза меньше |
| Побочные лепестки | −13 дБ | −31 дБ |

---

## 3) Сравнение с периодическим сигналом

Периодическая синусоида $ x(t) = \sin(2\pi f_0 t) $ имеет **дискретный спектр**:  
$$
X(f) = \frac{1}{2j}[\delta(f - f_0) - \delta(f + f_0)].
$$

Дельты $\pm f_0$ из бесконечной синусоиды превращаются в копии спектра окна, сосредоточенные в $\pm f_0$
В результате в спектре наблюдаются **размытые пики** около $\pm f_0$, форма которых определяется типом окна:
- прямоугольное окно: узкий пик, но сильные побочные лепестки;  
- окно Ханна: более широкий пик, но побочные лепестки значительно слабее.


## Задача 2.4. Спектр пачки равноотстоящих импульсов

|    Варианты:    |  1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  |  9  |  10 |  11 |  12 |
|:---------------:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|        N        |  3  |  4  |  3  |  4  |  3  |  4  |  3  |  4  |  3  |  4  |  3  |  4  |
| $\tau$**, мкс**     | 300 | 50  | 100 | 125 | 250 | 60  | 200 | 400 | 500 | 150 | 450 | 100 |
| $T_{period}$**, мкс** | 600 | 125 | 200 | 275 | 500 | 100 | 400 | 700 | 800 | 300 | 810 | 200 |

Ниже приведен пример вычисления спектр пачки из $N=4$ равноотстоящих импульсов длительностью $\tau = 100$ мкс с периодом повторения $T_{period}=400$ мкс.

Повторить вычисления для заданных значений $N$, $\tau$, $T_{period}$. Сравнить получившийся график с аналитическим выражением для спектра.


In [ ]:
N=3
tau=100e-6 #100 мкс
period = 400e-6 

f_band=np.linspace(-4/tau, 4/tau, 2000) # 2000 - число точек в диапазоне, в которых вычисляется X(f)

def boxcar(t, tau):
    
    for n in range(N):
        tau0 = period * n
        tau1 = tau + period * n
        if tau0 < t < tau1:
            return 0.1 # 0.1 В
    return 0.0 # 0 В
    
t_band=np.linspace(-0.5*tau, N*period, 1024)
plt.figure(figsize=[8, 4])
plt.plot(t_band*1e6, [boxcar(t, tau) for t in t_band])
plt.xlabel("Время $t$, мкс")
plt.ylabel("$x(t)$, В")
plt.title("Сигнал")
plt.tight_layout() 
plt.grid()
plt.show()

plt.figure(figsize=[8, 4])
plt.plot(f_band/1e3, fourier_transform(signal=boxcar, f_band=f_band, tau=tau, t1=0, t2=N*period, res_type="abs")*1e6)
plt.title("Спектр")
plt.xlabel("Частота $f$, кГц")
plt.ylabel("$|X(f)|$,  мкВ / Гц")
plt.tight_layout() 
plt.grid()
plt.show()

Код для построения графиков:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 3
tau = 100e-6
period = 400e-6
A = 0.1 

f_band = np.linspace(-4/tau, 4/tau, 2000)

dt = tau / 250.0
t = np.arange(0.0, N * period, dt)

x = np.zeros_like(t)
for n in range(N):
    start = n * period
    stop = start + tau
    x += A * ((t >= start) & (t < stop))

exp_mat = np.exp(-2j * np.pi * np.outer(f_band, t))
Xnum = exp_mat.dot(x) * dt

sinc_part = np.abs(np.sinc(f_band * tau))
den = np.sin(np.pi * f_band * period)
num = np.sin(np.pi * f_band * N * period)

ratio = np.where(np.isclose(den, 0.0, atol=1e-16), N, np.abs(num / den))

Xanal_mag = A * tau * sinc_part * ratio

Xnum_mag = np.abs(Xnum)

eps = 1e-20
rel_err = np.max(np.abs(Xnum_mag - Xanal_mag) / np.maximum(Xanal_mag, eps))

plt.figure(figsize=(8, 3.5))
plt.plot(t * 1e6, x)
plt.xlabel("Время $t$, мкс")
plt.ylabel("$x(t)$, В")
plt.title(f"Временный сигнал: N={N}, τ={tau*1e6:.0f} мкс, T={period*1e6:.0f} мкс")
plt.grid()
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 3.5))
plt.plot(f_band / 1e3, Xnum_mag, label="Численный |X(f)|")
plt.plot(f_band / 1e3, Xanal_mag, label="Аналитический |X(f)|", linestyle="--")
plt.xlabel("Частота $f$, кГц")
plt.ylabel("$|X(f)|$, В·c")
plt.title("Сравнение спектров (линейная шкала)")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


Как видно из графиков, аналитическое решение и вычислительное совпали хорошо.

# Занятие 3. Эффект наложения спектров при дискретизации сигналов.

## Задача 3.1. Эффект наложения при дискретизации прямоугольного импульса

|  Варианты:  |  1  |  2  |   3  |   4  |  5  |   6  |  7  |   8  |   9  |  10 |  11  |  12  |
|:-----------:|:---:|:---:|:----:|:----:|:---:|:----:|:---:|:----:|:----:|:---:|:----:|:----:|
| $\tau$**, мкс** | 100 | 500 | 1000 | 5000 | 400 | 2000 | 300 | 1500 | 3000 | 800 | 3500 | 4000 |


Рассмотреть прямоугольный импульс длительностью $1.05\tau$. Дискретизовать его с частотой $f_s  = 10/ \tau $.  Построить спектр прямоугольного импульса и полученного дискретизованного сигнала. 

Пронаблюдать наложение спектра при дискретизации. 

Найти сигнал,  который дискретизуется такой же последовательностью, как и прямоугольный импульс, но при этом не наблюдается эффект наложения (сигнал, который восстанавливается с помощью интерполяционного ряда Котельникова).

In [ ]:
def DTFT_abs3(xk, fs, M=2048):
    res=abs(np.fft.fftshift(np.fft.fft(xk, M)))
    return fs*np.arange(-1.5, 1.5, 1/M), np.hstack((res, res, res)) 

In [ ]:
tau=200*1e-6 #200 мкс
fs=10/tau

f_band=np.linspace(-1.5*fs, 1.5*fs, 1000) # 1000 - число точек в диапазоне, в которых вычисляется X(f)

def boxcar(t, tau):
    if 0 <= t <= 1.05*tau:
        return 1.0 
    elif np.isclose(t, 1.05*tau, atol=1e-12) or np.isclose(t, 0, atol=1e-12):
        return 1.0 
    else:
        return 0.0 

def sinc_phi(t, k, fs):
    dt=1/fs
    if np.isclose(0, t-k*dt):
        return 1.0
    else:
        return np.sin(np.pi*fs*(t-k*dt))/(np.pi*fs*(t-k*dt))
    
def sinc_interp(t, xk):
    return sum([xk[k]*fs*sinc_phi(t, k, fs) for k in range(len(xk))])

tk = np.arange(0, 2*tau+1.0/fs, 1.0/fs, dtype=float)
xk = np.array([(1/fs)*boxcar(tk1, tau) for tk1 in tk])
    
t_band=np.linspace(-0.2*tau, 2*tau, 1024)
plt.figure(figsize=[8, 4])
plt.plot(t_band*1e6, [boxcar(t, tau) for t in t_band], 'g', label='прямоугольный импульс')
plt.stem(tk*1e6, xk*fs, linefmt='b', markerfmt='bo', label='отсчеты сигнала $x[k]/{\\Delta t}$')



plt.plot(t_band*1e6, np.array([sinc_interp(t, xk) for t in t_band]), 'c', label='сигнал, востановленный \nрядом Котельникова')

plt.title("Сигнал")
plt.xlabel("Время $t$, мкс")
plt.ylabel("$x(t)$, В")
plt.grid()
plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.tight_layout() 
plt.show()

plt.figure(figsize=[8, 4])


DFTF_plot=DTFT_abs3(xk, fs)
plt.plot(DFTF_plot[0]/1e3, DFTF_plot[1]*1e6, 'b', label='ДВПФ для $x[k]$')

plt.plot(f_band/1e3, fourier_transform(signal=boxcar, f_band=f_band, tau=tau, t1=-2*tau, t2=2*tau, res_type="abs")*1e6, 
         'g', label='спектр\nпрямоугольного\nимпульса')

plt.title("Спектр")
plt.xlabel("Частота $f$, кГц")
plt.ylabel("$|X(f)|$,  мкВ / Гц")
plt.grid()
plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.tight_layout() 
plt.show()

## Задача 3.2. Эффект наложения при прореживании сигнала

| Варианты: | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10 | 11 | 12 |
|:---------:|----|----|----|----|----|----|----|----|----|----|----|----|
| $K_1$     | 35 | 10 | 20 | 25 | 15 | 40 | 20 | 15 | 40 | 35 | 10 | 25 |
| $K_2$     | 70 | 60 | 65 | 80 | 75 | 90 | 85 | 60 | 95 | 80 | 70 | 75 |


Выполнить прореживание (downsampling) сигнала камертона из файла `tuning-fork.wav` (основная частота сигнала 440 Гц, частота дискретизации 44100 Гц) без предварительной фильтрации, оставляя в одном случае каждый $K_1$-ый, а в другом каждый $K_2$-ый отсчет сигнала. В чем принципиальное отличие между спектрами получившихся сигналов и какими эффектами оно обусловлено? 

In [ ]:
def DTFT_abs(xk, fs, M=2**17):
    res=abs(np.fft.fftshift(np.fft.fft(xk, M)))
    return fs*np.arange(-0.5, 0.5, 1/M), np.hstack((res)) 

In [ ]:
fs, x=scipy.io.wavfile.read('tuning-fork.wav')
#for mono channel 
#x=x[:,1] 

plt.figure(figsize=[8, 3], dpi=100)

DFTF_plot=DTFT_abs(x/fs, fs)
plt.plot(DFTF_plot[0], DFTF_plot[1])
plt.grid()
plt.title("ДВПФ исходного сигнала")
plt.xlabel("Частота $f$, Гц")
plt.ylabel("$|X(f)|$")
plt.tight_layout() 

# downsampling
plt.figure(figsize=[8, 3], dpi=100)
K1=40
y=x[::K1]
DFTF_plot=DTFT_abs(y/(fs/K1), fs/K1)
plt.plot(DFTF_plot[0], DFTF_plot[1])
plt.grid()
plt.title("ДВПФ прореженного сигнала $К_1=$%i" %K1)
plt.xlabel("Частота $f$, Гц")
plt.ylabel("$|X(f)|$")
plt.tight_layout()
plt.show()

# downsampling
plt.figure(figsize=[8, 3], dpi=100)
K2=80
y=x[::K2]
    
DFTF_plot=DTFT_abs(y/(fs/K2), fs/K2)
plt.plot(DFTF_plot[0], DFTF_plot[1])
plt.grid()
plt.title("ДВПФ прореженного сигнала $К_2=$%i" %K2)
plt.xlabel("Частота $f$, Гц")
plt.ylabel("$|X(f)|$")
plt.tight_layout() 
plt.show()